# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
import sys
!{sys.executable} -m pip install pymongo

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.8/libexec/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
# Your Code
babelgum = db.companies.find({'name':{'$regex':'^[Bb]abelgum'}}, {'name':1, '_id':0})
babelgum

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [4]:
# Your Code
mt_fivek = db.companies.find({'number_of_employees':{'$gt':5000}}).limit(20).sort([('number_of_employees',-1)])

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [5]:
# Your Code
founded = db.companies.find({'founded_year': {'$gte':2000, '$lte':2005}}, {'name':1, 'founded_year':1, '_id':0}).sort([('founded_year', 1)])
founded[0]

{'name': 'AllofMP3', 'founded_year': 2000}

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [6]:
# Your Code
old_gold = db.companies.find({'ipo.valuation_amount':{'$gt':100000000}, 'founded_year':{'$lt':2010, '$ne':None}}, {'name':1, 'ipo':1, '_id':0}).sort([('ipo.valuation_amount',-1)])
old_gold[0]

{'name': 'GREE',
 'ipo': {'valuation_amount': 108960000000,
  'valuation_currency_code': 'JPY',
  'pub_year': 2008,
  'pub_month': 12,
  'pub_day': 17,
  'stock_symbol': '3632'}}

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [7]:
# Your Code
thnd_empl = db.companies.find({'number_of_employees':{'$lt':1000, '$ne':None}, 'founded_year':{'$lt':2005, '$ne':None}}).sort([('number_of_employees', -1)]).limit(10)

### 6. All the companies that don't include the `partners` field.

In [8]:
# Your Code
no_partners = db.companies.find({'partners':{'$exists':False}})
no_partners
# Esta no sé si es 'trampa', lo he hecho tal como dice la documentación 
# (https://docs.mongodb.com/manual/tutorial/query-for-null-fields/) y me sale que todos tienen el field

### 7. All the companies that have a null type of value on the `category_code` field.

In [9]:
# Your Code
no_category = db.companies.find({'category_code':None})

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [10]:
# Your Code
hund_thnd = db.companies.find({'number_of_employees':{'$gte':100, '$lt':1000}}, {'name':1, 'number_of_employees':1, '_id':0})
hund_thnd[0]

{'name': 'AdventNet', 'number_of_employees': 600}

### 9. Order all the companies by their IPO price in a descending order.

In [11]:
# Your Code
ipo_desc = db.companies.find({'ipo':{'$ne':None}}).sort([('ipo.valuation_amount', -1)])

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [12]:
# Your Code
more_empl = db.companies.find({}, {'name':1, 'number_of_employees':1, '_id':0}).sort([('number_of_employees', -1)]).limit(10)

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [13]:
# Your Code
second_sem = db.companies.find({'founded_month':{'$gte':6}}).limit(1000)

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [14]:
# Your Code
bf_sxix = db.companies.find({'founded_year':{'$lt':2000, '$ne':None}, 'acquisition.price_amount':{'$gt':10000000}})

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [15]:
# Your Code
acquired = db.companies.find({'acquisition.acquired_year':{'$gt':2010}}, {'name':1, 'acquisition':1, '_id':0}).sort([('acquisition.price_amount', -1)])
acquired[0]

{'name': 'T-Mobile',
 'acquisition': {'price_amount': 39000000000,
  'price_currency_code': 'USD',
  'term_code': None,
  'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
  'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
  'acquired_year': 2011,
  'acquired_month': 3,
  'acquired_day': 20,
  'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}}

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [16]:
# Your Code
by_year = db.companies.find({'founded_year':{'$ne':None}}, {'name':1, 'founded_year':1, '_id':0}).sort([('founded_year',1)])
by_year[0]

{'name': 'Alstrasoft', 'founded_year': 1800}

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [17]:
# Your Code
f_seven = db.companies.find({'$and': [{'founded_day':{'$lte':7}}, {'founded_day':{'$ne':None}}]}).sort([('acquisition.price_amount',-1)]).limit(10)

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [18]:
# Your Code
web = db.companies.find({'category_code':'web', 'number_of_employees':{'$gt':4000}}).sort([('number_of_employees', 1)])

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [19]:
# Your Code
eur = db.companies.find({'acquisition.price_amount':{'$gt':10000000}, 'acquisition.price_currency_code':'EUR'})

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [20]:
# Your Code
f_three_m = db.companies.find({'acquisition.acquired_month':{'$lte':3}}, {'name':1, 'acquisition':1, '_id':0}).limit(10)
f_three_m[0]

{'name': 'Kyte',
 'acquisition': {'price_amount': None,
  'price_currency_code': 'USD',
  'term_code': None,
  'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
  'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
  'acquired_year': 2011,
  'acquired_month': 1,
  'acquired_day': 31,
  'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}}

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [21]:
# Your Code
bonus_f = db.companies.find({
                                '$and':[
                                    {'founded_year':{'$gte':2000, '$lte':2010}},
                                    {'acquisition.acquired_year':{'$gt':2011}}]})

### 20. All the companies that have been 'deadpooled' after the third year.

In [22]:
# Your Code
bonus_s = db.companies.find({'$where': '[this.deadpooled_year - this.founded_year] > 3', 
                             'founded_year':{'$ne':None},
                             'deadpooled_month':{'$ne':None}}) 

for e in bonus_s:
    if (e['deadpooled_year'] - e['founded_year']) < 3:
        print(e)
    else:
        pass

print('All right!')

All right!
